In [1]:
import pandas as pd 
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

1. webdriver를 오픈해서 'https://dart.fss.or.kr/' 요청을 보낸다. 
2. id가 'textCrpNm2' 태그를 선택 
3. 특정 입력값을 보낸다 (send_key()) -> ENTER 이벤트 발생
4. 해당 페이지는 table태그가 1개 존재 
    - 해당 페이지의 html 소스코드를 문자화
    - pandas 에 있는 read_html() 함수에 인자로 사용
    - read_html() 결과에서 첫번째 데이터프레임을 변수에 저장
5. 페이지 소스 코드를 BeautifulSoup으로 파싱
6. tbody 태그 중 id가 'tbody'인 태그를 선택 
7. tr 태그 들을 모두 검색한다. 
8. 각각 tr 태그에서 a태그를 모두 검색하고 두번째 a태그의 href 속성의 값을 추출
9. base_url과 href 속성의 값을 더해서 새로운 리스트에 추가 
10. read_html() 함수를 사용해서 나온 데이터프레임에 열 결합

In [83]:
# 기본 주소를 변수에 대입
base_url = 'https://dart.fss.or.kr'

In [47]:
# 웹 브라우저 오픈
driver = webdriver.Chrome()

In [48]:
# 웹 브라우저에 특정 주소를 입력
driver.get(base_url)

In [49]:
# id가 textCrpNm2 인 태그를 선택 
input_tag = driver.find_element( By.ID, 'textCrpNm2' )

In [50]:
# input_tag에 특정 데이터를 입력 
input_tag.send_keys('삼성전자')

In [51]:
# input_tag ENTER 이벤트 발생
input_tag.send_keys(Keys.ENTER)

In [8]:
# 해당 페이지의 페이지소스를 이용해서 데이터프레임 생성
# 페이지소스를 문자화
html_data = str(driver.page_source)
# 문자화된 페이지 소스를 read_html() 인자로 사용
# 결과의 첫번째 데이터프레임을 변수에 대입
df = pd.read_html(html_data)[0]

C:\Users\moons\AppData\Local\Temp\ipykernel_20672\3305765445.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html_data)[0]


In [ ]:
df.head()

In [10]:
# 해당 페이지소스를 BeautifulSoup으로 파싱 
soup = bs(driver.page_source, 'html.parser')

In [12]:
# soup에서 tbody 태그 중 id가 tbody인 태그를 선택 
tbody_data = soup.find(
    'tbody', attrs={
        'id' : 'tbody'
    }
)

In [14]:
tr_list = tbody_data.find_all('tr')

In [19]:
# tr_list의 첫번째 데이터를 이용해서 반복 실행하는 구문을 테스트
base_url + tr_list[0].find_all('a')[1]['href']

'https://dart.fss.or.kr//dsaf001/main.do?rcpNo=20250604800428'

In [20]:
url_list = []
for tr_data in tr_list:
    url = base_url + tr_data.find_all('a')[1]['href']
    url_list.append(url)

In [ ]:
url_list

In [23]:
# df에 url_list 새로운 컬럼(url)에 대입 
df['url'] = url_list

In [ ]:
df.head()

In [25]:
# df를 csv로 저장 
df.to_csv('삼성전자.csv', index=False)

- 위의 코드들을 함수화
    - 첫번째 함수 
        - 매개변수 1개
            - driver : 웹 브라우져
        - table 태그를 찾아서 데이터프레임화
        - url 찾아서 데이터프레임에 대입 
        - 데이터프레임을 리턴

In [34]:
def create_df(web):
    # 해당 페이지의 페이지소스를 이용해서 데이터프레임 생성
    # 페이지소스를 문자화
    html_data = str(web.page_source)
    # 문자화된 페이지 소스를 read_html() 인자로 사용
    # 결과의 첫번째 데이터프레임을 변수에 대입
    df = pd.read_html(html_data)[0]
    # 해당 페이지소스를 BeautifulSoup으로 파싱 
    soup = bs(web.page_source, 'html.parser')
    # soup에서 tbody 태그 중 id가 tbody인 태그를 선택 
    tbody_data = soup.find(
        'tbody', attrs={
            'id' : 'tbody'
        }
    )
    tr_list = tbody_data.find_all('tr')
    url_list = []
    for tr_data in tr_list:
        url = base_url + tr_data.find_all('a')[1]['href']
        url_list.append(url)
    # df에 url_list 새로운 컬럼(url)에 대입 
    df['url'] = url_list
    # 데이터프레임을 리턴
    return df
    

In [ ]:
create_df(driver)

- 두번째 함수 
    - 매개변수 1개 
        - 종목의 이름을 입력하는 공간 
    - 웹브라우져 오픈 
    - 다트에 접속
    - 종목을 입력하고 검색
    - create_df()함수 호출 
    - 함수의 결과를 저장 
    - 파일의 이름을 print() 출력

In [30]:
import time

In [84]:
def dart_save(_name, _cnt = 1):
    # 기본 주소를 변수에 대입
    base_url = 'https://dart.fss.or.kr'
    driver = webdriver.Chrome()
    driver.get(base_url)
    time.sleep(1)
    input_tag = driver.find_element(By.ID , 'textCrpNm2')
    input_tag.send_keys(_name)
    input_tag.send_keys(Keys.ENTER)
    time.sleep(1)

    # 비어있는 데이터프레임을 생성 
    result = pd.DataFrame()
    for idx in range(_cnt):
        # class가 pageSkip 태그를 선택 
        skip_tag = driver.find_element(By.CLASS_NAME, 'pageSkip')
        # skip_tag에서 li 모두 찾아서 3번째부터 마지막 2번째 전까지 
        li_list = skip_tag.find_elements(By.TAG_NAME, 'a')[2:-2]
        li_list[idx].click()
        df = create_df(driver)
        # result와 df를 단순 행 결합 
        result = pd.concat( [result, df], axis=0 )
        time.sleep(1)
    result.to_csv(f"{_name}.csv", index=False)
    print(f"{_name}.csv 파일 생성 완료")
    driver.close()

In [85]:
dart_save('SK하이닉스', 3)

C:\Users\moons\AppData\Local\Temp\ipykernel_20672\483255769.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html_data)[0]
C:\Users\moons\AppData\Local\Temp\ipykernel_20672\483255769.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html_data)[0]
C:\Users\moons\AppData\Local\Temp\ipykernel_20672\483255769.py:7: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html_data)[0]


SK하이닉스.csv 파일 생성 완료


1. driver에서 class가 pageSkip 태그를 선택
2. li 태그들을 모두 찾는다. -> 3번째 부터 마지막에서 2번째 전까지데이터를 저장 
3. 함수에서 몇번 페이지를 이동할것인가 매개변수로 받아온다. 
4. 해당 변수의 값에 따라서 페이지를 이동 -> 해당 페이지의 데이터프레임을 불러온다. 
    - 각각 불러온 데이터프레임을 단순한 행 결합


In [62]:
# class가 pageSkip 태그를 선택 
skip_tag = driver.find_element(By.CLASS_NAME, 'pageSkip')
# skip_tag에서 li 모두 찾아서 3번째부터 마지막 2번째 전까지 
li_list = skip_tag.find_elements(By.TAG_NAME, 'a')[2:-2]

In [61]:
li_list[3].click()

In [86]:
# SK하이닉스.csv 로드 
df = pd.read_csv("SK하이닉스.csv")

In [ ]:
df.head()

In [87]:
# df 에서 보고서명 컬럼의 데이터가 '기업지배구조보고서공시' 인 
# 데이터만 필터링

# 조건식
flag = df['보고서명'] == '기업지배구조보고서공시'

url = df.loc[flag, 'url'][0]

In [74]:
import requests

In [75]:
res = requests.get(url)

In [76]:
soup2 = bs(res.text, 'html.parser')

In [79]:
soup2.find('iframe')

<iframe frameborder="0" id="ifrm" onload="iframeLoaded();" scrolling="auto" style="width: 100%; height : 100%; margin: 0; padding: 0" title="tailFrame"></iframe>

In [91]:
driver = webdriver.Chrome()

In [88]:
url

'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20250530800841'

In [92]:
driver.get(url)

In [93]:
soup3 = bs(driver.page_source, 'html.parser')

In [97]:
rep_url = base_url + soup3.iframe['src']

In [98]:
rep_res = requests.get(rep_url)

In [99]:
rep_soup = bs(rep_res.text, 'html.parser')

In [100]:
len(
    rep_soup.find_all('table')
)

348

In [106]:
[1,2].extend(   ['a', 'b'] )

In [ ]:
rep_soup.find('table')

In [ ]:
# 해당 보고서 중 테이블 태그에 데이터가 없는 경우가 존재 
# 데이터가 없는 테이블은 예외 처리 
# 데이터가 있는 테이블만 리스트에 추가 
table_list = rep_soup.find_all('table')

tables =[]
cnt = 0
for table_data in table_list:
    try:
        dfs = pd.read_html( str(table_data) )
    except:
        cnt+=1
        continue
    tables += dfs

In [111]:
cnt

10

In [112]:
len(tables)

357